In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

IMAGES_DIR = Path(r"G:\Other computers\Dell G3\Universitat\2nd Semester\MLMM\New\Images")
STRESS_CSV = Path(r"G:\Other computers\Dell G3\Universitat\2nd Semester\MLMM\New\StressValsreal.csv")
OUT_CSV    = Path("labels.csv")

# Load CSV
df = pd.read_csv(STRESS_CSV, sep=";", engine="python")
df = df.rename(columns=lambda c: c.strip())  # trim spaces from headers

# Rename stress column
if "25" in df.columns:
    df = df.rename(columns={"25": "stress"})
elif "Stress" in df.columns:
    df = df.rename(columns={"Stress": "stress"})

# Find class columns (Class 0..3)
class_cols = sorted([c for c in df.columns if c.lower().startswith("class")],
                    key=lambda x: int(re.sub(r"\D", "", x) or "-1"))
df["bracket"] = np.argmax(df[class_cols].to_numpy(), axis=1).astype(int)

print("Detected class columns:", class_cols)
print("Sample rows:")
print(df[["stress", "bracket"]].head())

# Scan image files and natural-sort
exts = {".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff"}
imgs = [p for p in IMAGES_DIR.iterdir() if p.suffix.lower() in exts]
assert imgs, f"No images found in {IMAGES_DIR}"

def extract_idx(path):
    m = re.search(r"(\d+)(?=\.[^.]+$)", path.name)
    if not m:
        raise ValueError(f"Cannot extract numeric id from filename: {path.name}")
    return int(m.group(1))

imgs_sorted = sorted(imgs, key=extract_idx)

print("First 5 images after sorting:",
      [p.name for p in imgs_sorted[:5]])

# Align by order & create labels.csv
assert len(imgs_sorted) == len(df), \
    f"Counts differ: {len(imgs_sorted)} images vs {len(df)} CSV rows. Check folder/CSV."

labels = pd.DataFrame({
    "filename": [p.name for p in imgs_sorted],
    "stress":   df["stress"].astype(float).to_numpy(),
    "bracket":  df["bracket"].to_numpy()
})

labels.to_csv(OUT_CSV, index=False)
print(f"\nSaved {len(labels)} rows to {OUT_CSV.resolve()}")
print(labels.head(6))


In [ ]:
from pathlib import Path
import pandas as pd

OUT_CSV = Path(r"G:\Other computers\Dell G3\Universitat\2nd Semester\MLMM\New\labels.csv")

labels.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
print("Saved to:", OUT_CSV)
print("Exists?", OUT_CSV.exists(), " Size (bytes):", OUT_CSV.stat().st_size if OUT_CSV.exists() else "N/A")

print(pd.read_csv(OUT_CSV).head())
